### Resale Transactions Preprocessing

In [18]:
import pandas as pd
import os

Get directories of each dataset

In [19]:
current_directory = os.getcwd()
data_folder = 'resale_transactions'
folder_path = os.path.join(current_directory, data_folder)
dataset_names = os.listdir(folder_path)

Read each file into a separate dataframe

In [20]:
dfs = []
for i in range(len(dataset_names)):
    filepath = os.path.join(folder_path, dataset_names[i])
    dfs.append(pd.read_csv(filepath))

Examine Consistency of Fields across datasets, using 1990 as the benchmark (least fields)
- Most fields are consistent, with the exception of flat model and resale price
- Flat model changes according to time period, can be dropped since flat type field is present
- Resale prices come in the form of floats and integers, can cast to float

In [21]:
for col in dfs[1].columns.tolist():
    print(f"\nExamining {col} unique values across historical data\n") 
    for i in range(len(dfs)):
        print(sorted(dfs[i][col].unique()))


Examining month unique values across historical data

['2012-03', '2012-04', '2012-05', '2012-06', '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12']
['1990-01', '1990-02', '1990-03', '1990-04', '1990-05', '1990-06', '1990-07', '1990-08', '1990-09', '1990-10', '1990-11', '1990-12', '1991-01', '1991-02', '1991-03', '1991-04', '1991-05', '1991-06', '1991-07', '1991-08', '1991-09', '1991-10', '1991-11', '1991-12', '1992-01', '1992-02', '1992-03', '1992-04', '1992-05', '1992-06', '1992-07', '1992-08', '1992-09', '1992-10', '1992-11', '1992-12', '1993-01', '1993-02', '1993-03', '1993-04', '1993-05', '1993-06', '1993-07', '1993-08', '1993-09', '1993-10', '1993-11', '1993-12', '1994-01', '1994-02', '1994-03', '1994-04

Modify remaining_lease, combine dataframes and cast resale_prices to float (auto casted). Separate month and year values.

In [22]:
for i in range(len(dfs)):
    dfs[i]['remaining_lease'] = dfs[i]['lease_commence_date'] + 99 - dfs[i]['month'].str.split('-').str[0].astype(int)
    dfs[i]['year'] = dfs[i]['month'].str.split('-').str[0].astype(int)
    dfs[i]['month'] = dfs[i]['month'].str.split('-').str[1].astype(int)
combined_df = pd.concat([dfs[i] for i in range(len(dfs))])
combined_df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year
0,3,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,73,2012
1,3,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,67,2012
2,3,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,67,2012
3,3,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,71,2012
4,3,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,67,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
37148,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,83,2016
37149,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,70,2016
37150,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,75,2016
37151,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,71,2016


Generate csv of complete transactions (~800k)

In [23]:
combined_df.to_csv('resale_transactions_complete.csv', index=False)

Testing of API against existing entries
- Check if offsetting against no of transactions after 2017 works

In [28]:
df = pd.read_csv('resale_transactions_complete.csv')
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year
0,3,ANG MO KIO,2 ROOM,172,ANG MO KIO AVE 4,06 TO 10,45.0,Improved,1986,250000.0,73,2012
1,3,ANG MO KIO,2 ROOM,510,ANG MO KIO AVE 8,01 TO 05,44.0,Improved,1980,265000.0,67,2012
2,3,ANG MO KIO,3 ROOM,610,ANG MO KIO AVE 4,06 TO 10,68.0,New Generation,1980,315000.0,67,2012
3,3,ANG MO KIO,3 ROOM,474,ANG MO KIO AVE 10,01 TO 05,67.0,New Generation,1984,320000.0,71,2012
4,3,ANG MO KIO,3 ROOM,604,ANG MO KIO AVE 5,06 TO 10,67.0,New Generation,1980,321000.0,67,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
893822,12,YISHUN,5 ROOM,297,YISHUN ST 20,13 TO 15,112.0,Improved,2000,488000.0,83,2016
893823,12,YISHUN,5 ROOM,838,YISHUN ST 81,01 TO 03,122.0,Improved,1987,455000.0,70,2016
893824,12,YISHUN,EXECUTIVE,664,YISHUN AVE 4,10 TO 12,181.0,Apartment,1992,778000.0,75,2016
893825,12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,71,2016


In [29]:
txs_2023 = df[df['year'] >= 2017]
txs_2023.sort_values(by='month', ascending=False)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year
401959,12,BEDOK,5 ROOM,140,BEDOK NTH ST 2,13 TO 15,120.0,Improved,1998,608000.0,78,2019
424629,12,BEDOK,3 ROOM,610,BEDOK RESERVOIR RD,01 TO 03,67.0,New Generation,1982,275000.0,61,2020
426010,12,PUNGGOL,5 ROOM,204D,PUNGGOL FIELD,01 TO 03,110.0,Improved,2004,455000.0,83,2020
426011,12,PUNGGOL,5 ROOM,128D,PUNGGOL FIELD WALK,01 TO 03,110.0,Improved,2003,410000.0,82,2020
426012,12,PUNGGOL,5 ROOM,128C,PUNGGOL FIELD WALK,04 TO 06,110.0,Improved,2003,415000.0,82,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
405336,1,WOODLANDS,4 ROOM,848,WOODLANDS ST 82,01 TO 03,93.0,New Generation,1995,310000.0,74,2020
405335,1,WOODLANDS,4 ROOM,409,WOODLANDS ST 41,07 TO 09,106.0,Model A,1996,345000.0,75,2020
405334,1,WOODLANDS,4 ROOM,429,WOODLANDS ST 41,04 TO 06,84.0,Simplified,1996,283000.0,75,2020
405333,1,WOODLANDS,4 ROOM,429,WOODLANDS ST 41,07 TO 09,84.0,Simplified,1996,295000.0,75,2020


In [25]:
txs_2023[(txs_2023['resale_price'] == 620000.0) & (txs_2023['town'] == 'WOODLANDS')]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,year
485105,1,WOODLANDS,5 ROOM,671,WOODLANDS DR 71,04 TO 06,111.0,Improved,2000,620000.0,76,2023
485128,1,WOODLANDS,5 ROOM,785D,WOODLANDS RISE,07 TO 09,113.0,Improved,2018,620000.0,94,2023
485140,1,WOODLANDS,5 ROOM,851,WOODLANDS ST 83,04 TO 06,135.0,Model A,1996,620000.0,72,2023
